In [57]:
import plotly.express as px
import streamlit as st
import requests
import pandas as pd
from translate_API_output import traducir, traducir_tweets, preprocess_tweet
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from deep_translator import GoogleTranslator
import re
import streamlit as st
import torch
from transformers import RobertaTokenizer
from custom_class_final_model import CustomRobertaModel
from model_load_apply import load_custom_sentiment_model, predict_sentiment, analyze_sentiments
from dashboard_charts import plot_wordcloud, sentiment_dist, format_data_model_output, obtain_summary, likes_over_words_amount, sentiment_dist_plotly, create_banner
import plotly.graph_objects as go
from PIL import Image
import numpy as np
import pprint
import json
from datetime import datetime

In [64]:
url = "https://threads-api4.p.rapidapi.com/api/search"

keyword = "Lebron"

querystring = {"query":str(keyword), "limit": '50'}

headers_threads = {
	"x-rapidapi-key": "2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a",
	"x-rapidapi-host": "threads-api4.p.rapidapi.com"
}

response = requests.get(url, headers=headers_threads, params=querystring)



In [49]:
pp = pprint.PrettyPrinter()
pp.pprint(response.json())

{'data': {'searchResults': {'edges': [{'cursor': '',
                                       'node': {'__typename': 'XDTSearchThread',
                                                'thread': {'id': '3484565590885440736',
                                                           'thread_header_context': None,
                                                           'thread_items': [{'line_type': 'none',
                                                                             'post': {'accessibility_caption': None,
                                                                                      'audio': None,
                                                                                      'caption': {'has_translation': None,
                                                                                                  'pk': '18333988588150915',
                                                                                                  'text': 'Both '
          

In [65]:
json_data = response.json()
json_data

{'message': "Endpoint '/api/search' does not exist"}

In [61]:
def extract_text_from_json(json_data):
    extracted_text = []
    try:
        # Acceder a la clave 'searchResults'
        search_results = json_data['data']['searchResults']

        # Iterar sobre 'edges'
        for edge in search_results.get('edges', []):
            thread = edge.get('node', {}).get('thread', {})

            # Asegurarse de que 'thread_items' existe antes de iterar
            thread_items = thread.get('thread_items', [])
            if not thread_items:
                print("Warning: No 'thread_items' found in some thread.")
                continue  # Si no hay 'thread_items', pasar al siguiente 'edge'

            # Iterar sobre 'thread_items'
            for item in thread_items:
                # Acceder a la clave 'post' y extraer el texto
                post = item.get('post', {})

                if 'caption' in post and 'text' in post['caption']:
                    full_text = post['caption']['text']

                    # Acceder a los datos del post
                    post_date = item['post']['taken_at']
                    post_date = datetime.utcfromtimestamp(post_date).strftime('%Y-%m-%d %H:%M:%S')
                    favorite_count = post.get('like_count', 0)


                    # Añadir la información extraída a la lista
                    extracted_text.append((post_date, full_text, favorite_count))

    except KeyError as e:
        print(f"Error: Key {str(e)} not found in JSON.")

    return extracted_text


In [62]:
extracted_text_list = extract_text_from_json(response.json())
print(extracted_text_list)

[('2024-10-22 15:56:50', 'Both lol. My favorite player is LeBron & he’s the reason I’m this big a fan of the sport honestly. \nAlso always root for home team Cleveland!', 0), ('2024-10-22 15:56:42', 'BREAKING NEWS: SHOCKING NEWS: LeBron James Admits He Slept with Diddy When He Was Young, Lured Into Wild Parties!\nRead more: https://buff.ly/3Nd2NUz', 0), ('2024-10-22 15:55:59', 'Brand new LETS GO LAKERS Podcast! We season opener against Minnesota and go back in time to the infamous preseason brawl with Rick Fox and Doug Christie. Link in bio!\n\n#lakers #lakerspodcast #letsgolakers #lebron #anthonydavis #basketballpodcast #losangeles', 0), ('2024-10-22 15:55:13', 'Did you forget that Kyrie and Love were out in 2015 and Lebron still took the Warriors to 6?  They win w a full squad.  And Bogut doesn’t compare to Kyrie or Love. Draymond took himself out of game 6 w dirty play.  Intentionally hitting opponents in the nuts will do that.', 0), ('2024-10-22 15:53:33', 'Embiid\nKD\nGiannis\nCur

C:\Program Files\KMSpico\temp\ipykernel_9356\632653392.py:27: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  post_date = datetime.utcfromtimestamp(post_date).strftime('%Y-%m-%d %H:%M:%S')


In [63]:
pd.DataFrame(extracted_text_list)

,0,1,2
0,2024-10-22 15:56:50,Both lol. My favorite player is LeBron & he’s ...,0
1,2024-10-22 15:56:42,BREAKING NEWS: SHOCKING NEWS: LeBron James Adm...,0
2,2024-10-22 15:55:59,Brand new LETS GO LAKERS Podcast! We season op...,0
3,2024-10-22 15:55:13,Did you forget that Kyrie and Love were out in...,0
4,2024-10-22 15:53:33,Embiid\nKD\nGiannis\nCurry\nAntman\nTatum\nLuk...,0
5,2024-10-22 15:50:06,Breaking: Caitlin Clark Turns Down Record-Brea...,2
6,2024-10-22 15:47:39,Alleged Leaked Tape Involving Diddy and LeBron...,0
7,2024-10-22 15:40:31,"Aww, my youngest brother is also a Lakers fan....",1
8,2024-10-22 15:33:58,D white o14 pts\nJrue o0.5 steals\nRandle u18....,0
